Analisis de los tamaños de los petido y MHC, con el objetivo de definir un tamaño maximo y hacer padding

In [4]:
import pandas as pd

data = pd.read_csv('../dataset/netMHCIIpan3.2/train.csv')
#print(data)

print("TRAIN")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"mean peptide length {data.peptide.str.len().mean()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")


data = pd.read_csv('../dataset/netMHCIIpan3.2/test.csv')
#print(data)

print("\nTEST")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")


data = pd.read_csv('../dataset/netMHCIIpan3.2/eval.csv')
#print(data)

print("\nVAL")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")

TRAIN
max peptide length 37
min peptide length 9
mean peptide length 15.313720551257678
max MHC length 34
min MHC length 34

TEST
max peptide length 37
min peptide length 9
max MHC length 34
min MHC length 34

VAL
max peptide length 37
min peptide length 9
max MHC length 34
min MHC length 34


In [1]:
import pandas as pd

data = pd.read_csv('dataset/hlab/hlab_train.csv')
#print(data)

print("TRAIN")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"mean peptide length {data.peptide.str.len().mean()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")


data = pd.read_csv('dataset/hlab/hlab_test.csv')
#print(data)

print("\nTEST")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")


data = pd.read_csv('dataset/hlab/hlab_val.csv')
#print(data)

print("\nVAL")
print(f"max peptide length {data.peptide.str.len().max()}")
print(f"min peptide length {data.peptide.str.len().min()}")
print(f"max MHC length {data.mhc.str.len().max()}")
print(f"min MHC length {data.mhc.str.len().min()}")

TRAIN
max peptide length 14
min peptide length 8
mean peptide length 9.462156250521781
max MHC length 34
min MHC length 34

TEST
max peptide length 14
min peptide length 8
max MHC length 34
min MHC length 34

VAL
max peptide length 14
min peptide length 8
max MHC length 34
min MHC length 34


# TAPE with Transformers

In [7]:
from transformers import Trainer, TrainingArguments, BertConfig
from tape import ProteinBertConfig

config1 = ProteinBertConfig.from_pretrained("bert-base", num_labels=2)
config2 = BertConfig.from_pretrained("../models/esm2_t30_150M_UR50D", num_labels=2)

print(config1)
print(config2)

You are using a model of type esm to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


{
  "attention_probs_dropout_prob": 0.1,
  "base_model": "transformer",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "input_size": 768,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 8192,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_size": 768,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "vocab_size": 30
}

BertConfig {
  "_name_or_path": "/tmp/facebook/esm2_t30_150M_UR50D",
  "architectures": [
    "EsmForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "emb_layer_norm_before": false,
  "esmfold_config": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 640,
  "initializer_range": 0.02,
  "intermediate_size": 2560,
  "is_folding_model": false,
  "layer_norm_eps": 1e-0

In [16]:
import torch.utils.checkpoint
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn import functional as F
from torch.nn import MSELoss

from tape import ProteinBertAbstractModel, ProteinBertModel

from transformers import BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.file_utils import add_start_docstrings, add_start_docstrings_to_model_forward
from transformers import Trainer, TrainingArguments, BertConfig
from transformers.models.bert.modeling_bert import BERT_INPUTS_DOCSTRING, BERT_START_DOCSTRING
import math
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_auc_score

@add_start_docstrings(
    """
    Bert Model transformer with a sequence classification/regression head on top (a linear layer on top of the pooled
    output) e.g. for GLUE tasks.
    """,
    BERT_START_DOCSTRING,
)
class TapeLinear(ProteinBertAbstractModel):

    def __init__(self, config):
        super().__init__(config)

        self.num_labels = config.num_labels

        self.bert = ProteinBertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    @add_start_docstrings_to_model_forward(BERT_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """

        
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )



In [31]:
import torch.utils.checkpoint
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn import functional as F
from torch.nn import MSELoss
from transformers import BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.file_utils import add_start_docstrings, add_start_docstrings_to_model_forward
from transformers.models.bert.modeling_bert import BERT_INPUTS_DOCSTRING, BERT_START_DOCSTRING
from transformers import Trainer, TrainingArguments, BertConfig

from tape.tokenizers import TAPETokenizer
from tape import ProteinBertConfig

from data_loader import My_Load_Dataset
from transformers import EarlyStoppingCallback, IntervalStrategy
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_auc_score

from dataloader import BertDataset
from torch.utils.data import DataLoader

import math

def compute_metrics(pred):
    labels = pred.label_ids
    prediction=pred.predictions
    preds = prediction.argmax(-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    precision = tp / (tp + fp) 
    recall = tp / (tp + fn)
    sn = tp / (tp + fp)       
    sp = tn / (tn + fp)  # true negative rate
    mcc = matthews_corrcoef(labels, preds)
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    f1 = 2 * (precision * recall) / (precision + recall)
    return {
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'sn': sn,
        'sp': sp,
        'accuracy': acc,
        'mcc': mcc
    }



trainset = BertDataset('../dataset/netMHCIIpan3.2/train_micro.csv', max_pep_len=24)
valset = BertDataset('../dataset/netMHCIIpan3.2/eval_micro.csv', max_pep_len=24)

train_data = DataLoader(        trainset,
                                batch_size=32,
                                shuffle=True,
                                num_workers=16,
                                pin_memory=True,
                                collate_fn=trainset.collate_fn)

val_data = DataLoader(        valset,
                              batch_size=64,
                              num_workers=16,
                              pin_memory=True,
                              collate_fn=valset.collate_fn)

model_name = "../models/esm2_t6_8M_UR50D"
train_dataset = My_Load_Dataset(path="../dataset/netMHCIIpan3.2/train_micro.csv", tokenizer_name=model_name, max_length=71)
val_dataset = My_Load_Dataset(path="../dataset/netMHCIIpan3.2/eval_micro.csv", tokenizer_name=model_name, max_length=71)

print(train_dataset[0])
print(train_dataset[0]['input_ids'].shape)
print(trainset[0])
print(trainset[0]['input_ids'].shape)

"""
print(train_dataset[0]['input_ids'].shape)
print(val_dataset[0])
print(test_dataset[0])

config = ProteinBertConfig.from_pretrained("bert-base", num_labels=2)
#config = BertConfig.from_pretrained("bert-base", num_labels=2)
config.rnn = "lstm"
config.num_rnn_layer = 2
config.rnn_dropout = 0.1
config.rnn_hidden = 768
config.length = 51
config.cnn_filters = 512
config.cnn_dropout = 0.1"""


{'input_ids': tensor([ 0, 16,  9, 18, 18, 12,  5,  8,  6,  5,  5,  7, 13,  5, 12, 20,  9,  8,
         8, 18, 13, 19, 18, 13, 12, 13,  9,  5, 11, 19, 21,  7,  7, 18, 11, 11,
        12, 14,  4,  7,  5,  4, 11,  4, 11,  8, 19,  4,  6,  4, 15, 24, 24, 24,
        24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,  2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'labels': tensor(1)}
torch.Size([71])
{'input_ids': array([ 2, 20,  9, 10, 10, 13,  5, 22, 11,  5,  5, 25,  8,  5, 13, 16,  9,
       22, 22, 10,  8, 28, 10,  8, 13,  8,  9,  5, 23, 28, 12, 25, 25, 10,
       23, 23, 13, 19, 15, 25,  5, 15, 23, 15, 23, 22, 28, 15, 11, 15, 14,
        3]), 'input_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 

'\nprint(train_dataset[0][\'input_ids\'].shape)\nprint(val_dataset[0])\nprint(test_dataset[0])\n\nconfig = ProteinBertConfig.from_pretrained("bert-base", num_labels=2)\n#config = BertConfig.from_pretrained("bert-base", num_labels=2)\nconfig.rnn = "lstm"\nconfig.num_rnn_layer = 2\nconfig.rnn_dropout = 0.1\nconfig.rnn_hidden = 768\nconfig.length = 51\nconfig.cnn_filters = 512\nconfig.cnn_dropout = 0.1'